<a href="https://colab.research.google.com/github/sneha87107/deep_learning_files_sneha/blob/main/Agentic_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup API Key Properly

In [ ]:
import os
from google.colab import userdata

os.environ["GROQ_API_KEY"] = userdata.get("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"] = userdata.get("TAVILY_API_KEY")

In [ ]:
!pip install -U langchain langgraph langchain-tavily langchain-groq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.9/43.9 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.7/216.7 kB 19.4 MB/s eta 0:00:00


In [ ]:
from langchain_groq import ChatGroq
from langchain_tavily import TavilySearch
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool

In [ ]:
search = TavilySearch()
search.invoke("What is current USD vs INR price")

{'query': 'What is current USD vs INR price',
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://dollarrupee.in/',
   'title': 'DOLLAR TO RUPEE TODAY AND FORECAST',
   'content': "Current USD to INR exchange rate equals 88.6570 Rupees per 1 Dollar. Today's range: 88.6570-88.6570. Yesterday's rate 88.6570. The change for today 0.0000",
   'score': 0.9310187,
   'raw_content': None},
  {'url': 'https://www.tradingview.com/symbols/USDINR/',
   'title': 'USD INR Chart – Dollar to Rupee Rate - TradingView',
   'content': 'The current rate of USDINR is 88.6300 INR — it has decreased by −0.10% in the past 24 hours. See more of USDINR rate dynamics on the detailed chart.',
   'score': 0.92825955,
   'raw_content': None},
  {'url': 'https://wise.com/us/currency-converter/usd-to-inr-rate',
   'title': 'US dollars to Indian rupees Exchange Rate. Convert USD/INR - Wise',
   'content': 'USD to INR exchange rates today ; 1 USD, 88.69 INR ; 5 USD, 443.43 INR ;

In [ ]:
@tool
def calculate_emi(p: str, n: str, r: str) -> dict[str, float]:
  """
  Inputs :
  p: principal in INR
  n: Number of years
  r: rate of interest in %p.a.
  Outputs :
  A dictionary containing emi, amount and interest
  """
  p = float(p)
  n = float(n)
  r = float(r)
  n = n*12
  r = r/1200
  x = (1 + r)**n
  emi = (p * r * x) / (x - 1)
  amt = emi * n
  interest = amt - p
  return {"emi": emi, "amount": amt, "interest": interest}

## Get the Model

In [ ]:
model = ChatGroq(model="llama-3.3-70b-versatile")
model.invoke("Hello How are you!")

AIMessage(content="Hello. I'm just a language model, so I don't have feelings or emotions like humans do, but I'm functioning properly and ready to help you with any questions or tasks you may have. How can I assist you today?", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 48, 'prompt_tokens': 40, 'total_tokens': 88, 'completion_time': 0.09587115, 'prompt_time': 0.001960935, 'queue_time': 0.054336304, 'total_time': 0.097832085}, 'model_name': 'llama-3.3-70b-versatile', 'system_fingerprint': 'fp_9e1e8f8435', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--aac5a1dc-aa9b-42a4-9f0b-7a83e9453433-0', usage_metadata={'input_tokens': 40, 'output_tokens': 48, 'total_tokens': 88})

In [ ]:
tools = [search, calculate_emi]
agent_executor = create_react_agent(model, tools, checkpointer=MemorySaver())

In [ ]:
# Config is required for remembering past conversation
config = {"configurable": {"thread_id": "abc_123"}}

input_message = {
    "role": "user",
    "content": "Provide the weather forecast for this week in Pune"
}

for step in agent_executor.stream(
    {"messages": input_message}, config, stream_mode="values"
):
  step["messages"][-1].pretty_print()

================================ Human Message =================================

Provide the weather forecast for this week in Pune
================================== Ai Message ==================================
Tool Calls:
  tavily_search (x6rbwd85d)
 Call ID: x6rbwd85d
  Args:
    query: Pune weather forecast this week
    time_range: week
    topic: general
================================= Tool Message =================================
Name: tavily_search

{"query": "Pune weather forecast this week", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Pune", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Pune', 'region': 'Maharashtra', 'country': 'India', 'lat': 18.5333, 'lon': 73.8667, 'tz_id': 'Asia/Kolkata', 'localtime_epoch': 1759059715, 'localtime': '2025-09-28 17:11'}, 'current': {'last_updated_epoch': 1759059000, 'last_updated': '2025-09-28 17:00', 'temp_c': 20.9, 'temp_f': 69.6, 'is_day': 1, 'condition': {

In [ ]:
def ai_agent(query: str, thread_id: str):
  config = {"configurable": {"thread_id": thread_id}}
  input_message = {
      "role": "user",
      "content": query
  }
  for step in agent_executor.stream(
      {"messages": input_message}, config, stream_mode="values"
  ):
    step["messages"][-1].pretty_print()

In [ ]:
thread_id = "def456"
ai_agent("How is banking stocks sentiment in India", thread_id=thread_id)

================================ Human Message =================================

How is banking stocks sentiment in India
================================== Ai Message ==================================
Tool Calls:
  tavily_search (28595gkqg)
 Call ID: 28595gkqg
  Args:
    query: banking stocks sentiment in India
    topic: finance
================================= Tool Message =================================
Name: tavily_search

{"query": "banking stocks sentiment in India", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.forbes.com/advisor/in/investing/best-penny-stocks/", "title": "Best Penny Stocks To Buy Now In India September 2025", "content": "This consistent financial performance, its strategic position in the banking sector, and its efforts to restore investor confidence make YES Bank an intriguing choice for those looking to invest in penny stocks with significant growth prospects. Given its strategic initiatives and market posi

In [ ]:
ai_agent(
    query = "Principal is 10_00_000 INR for 5 years. Rate of interest is 9% p.a, Calculate EMI",
    thread_id = thread_id
)

================================ Human Message =================================

Principal is 10_00_000 INR for 5 years. Rate of interest is 9% p.a, Calculate EMI
================================== Ai Message ==================================
Tool Calls:
  calculate_emi (f8xm3jc3c)
 Call ID: f8xm3jc3c
  Args:
    n: 5
    p: 1000000
    r: 9
================================= Tool Message =================================
Name: calculate_emi

{"emi": 20758.355226353873, "amount": 1245501.3135812324, "interest": 245501.31358123245}
================================== Ai Message ==================================

The EMI for a principal amount of 10,00,000 INR, for 5 years, at an interest rate of 9% p.a. is 20,758.36 INR. The total amount to be paid is 12,45,501.31 INR and the total interest paid is 2,45,501.31 INR.


## Gemini model

In [ ]:
!pip install -U langchain-google-genai


In [ ]:
import os

os.environ["GOOGLE_API_KEY"] = userdata.get("GOOGLE_API_KEY")

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash")
llm.invoke("What is 2+4")


AIMessage(content='2 + 4 = 6', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--3159fac1-a8f7-431f-904c-8757281186c2-0', usage_metadata={'input_tokens': 7, 'output_tokens': 30, 'total_tokens': 37, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 23}})

In [ ]:
tools = [search, calculate_emi]
agent_executor = create_react_agent(llm, tools, checkpointer=MemorySaver())


In [ ]:
def ai_agent(query: str, thread_id: str):
  config = {"configurable": {"thread_id": thread_id}}
  input_message = {
      "role": "user",
      "content": query
  }
  for step in agent_executor.stream(
      {"messages": input_message}, config, stream_mode="values"
  ):
    step["messages"][-1].pretty_print()

In [ ]:
ai_agent("Principal 10_00_000 , Number of years 5 , rate of interest 8.1 Calulate EMI",
         thread_id="def456")

================================ Human Message =================================

Principal 10_00_000 , Number of years 5 , rate of interest 8.1 Calulate EMI
================================== Ai Message ==================================
Tool Calls:
  calculate_emi (6eec3eaf-7e96-478f-ac4f-2ed726788483)
 Call ID: 6eec3eaf-7e96-478f-ac4f-2ed726788483
  Args:
    r: 8.1
    n: 5
    p: 1000000
================================= Tool Message =================================
Name: calculate_emi

{"emi": 20324.28652922556, "amount": 1219457.1917535337, "interest": 219457.19175353367}
================================== Ai Message ==================================

Your EMI will be 20,324.29. The total amount payable will be 1,219,457.19, with a total interest of 219,457.19.


In [ ]:
ai_agent("How is sentiment for banking stocks in India",
         thread_id="def456")

================================ Human Message =================================

How is sentiment for banking stocks in India
================================== Ai Message ==================================
Tool Calls:
  tavily_search (40f25294-dea2-475c-9f34-78b9397abe04)
 Call ID: 40f25294-dea2-475c-9f34-78b9397abe04
  Args:
    query: sentiment for banking stocks in India
    topic: finance
================================= Tool Message =================================
Name: tavily_search

{"query": "sentiment for banking stocks in India", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.forbes.com/advisor/in/investing/stock-market-outlook/", "title": "India Stock Market Outlook For November (2025)", "content": "On the eve of Diwali trading, the Indian stock market indices experienced a positive uptick, reflecting a generally optimistic sentiment among", "score": 0.75442725, "raw_content": null}, {"url": "https://www.forbes.com/advisor/in

In [ ]:
ai_agent("Vonvert 100 USD to INR as per latest rates",
         thread_id="def456")

================================ Human Message =================================

Vonvert 100 USD to INR as per latest rates
================================== Ai Message ==================================
Tool Calls:
  tavily_search (47db1dda-8d4c-4985-b745-77ff415b4244)
 Call ID: 47db1dda-8d4c-4985-b745-77ff415b4244
  Args:
    query: 100 USD to INR latest rate
================================= Tool Message =================================
Name: tavily_search

{"query": "100 USD to INR latest rate", "follow_up_questions": null, "answer": null, "images": [], "results": [{"url": "https://www.xe.com/en-us/currencyconverter/convert/?Amount=100&From=USD&To=INR", "title": "100 USD to INR - US Dollars to Indian Rupees Exchange Rate - Xe", "content": "USD - US Dollar. To. INR - Indian Rupee. 100.00 US Dollar = 8,871.0143 Indian Rupee. 1 USD = 88.7101 INR. 1 INR = 0.0112727 USD. US Dollar to Indian Rupee", "score": 0.90985113, "raw_content": null}, {"url": "https://www.makemytrip.com/tripmon